In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk

In [2]:
df=pd.read_csv("Reviews.csv", encoding='utf-8')
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [5]:
df.shape

(568454, 10)

In [6]:
#sns.countplot(df['Score'])

In [3]:
#BASIC NLTK

nltk.download('punkt')
nltk.download('punkt_tab')  # Download the missing resource
nltk.download('averaged_perceptron_tagger')  # Download the missing resource
example=df['Text'][0]
print(example)
tokens=nltk.word_tokenize(example)
print(tokens)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\supep\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\supep\AppData\Roaming\nltk_data...


I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.
['I', 'have', 'bought', 'several', 'of', 'the', 'Vitality', 'canned', 'dog', 'food', 'products', 'and', 'have', 'found', 'them', 'all', 'to', 'be', 'of', 'good', 'quality', '.', 'The', 'product', 'looks', 'more', 'like', 'a', 'stew', 'than', 'a', 'processed', 'meat', 'and', 'it', 'smells', 'better', '.', 'My', 'Labrador', 'is', 'finicky', 'and', 'she', 'appreciates', 'this', 'product', 'better', 'than', 'most', '.']


[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\supep\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


#step 1 Using nltks sentiment intensitty analysxer
Uses bag of words approach
stop words are removed 
each word is scored and combined to a total score


VADER (Valence Aware Dictionary and sEntiment Reasoner) 

In [4]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sia=SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\supep\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [5]:
sia.polarity_scores(example)

{'neg': 0.0, 'neu': 0.695, 'pos': 0.305, 'compound': 0.9441}

In [7]:
sia.polarity_scores("book is bad")

{'neg': 0.636, 'neu': 0.364, 'pos': 0.0, 'compound': -0.5423}

📌 VADER Sentiment Analysis – Short Note

    VADER (Valence Aware Dictionary and sEntiment Reasoner) is a rule-based sentiment analysis tool from NLTK.

    It uses a predefined lexicon of words with positive/negative scores and rules for handling intensifiers, negations, punctuation, etc.

VADER Output:

{'neg': ..., 'neu': ..., 'pos': ..., 'compound': ...}

    neg, neu, pos: Proportions of negative, neutral, and positive words.

    compound: Overall sentiment score ranging from -1 (most negative) to +1 (most positive).

How compound is calculated:

    VADER sums all adjusted word scores to get a raw sentiment score.

    It then normalizes it using:
    compound=x/ root(x2+15)
    compound=x2+15

    ​x​

    (where xx is the raw score)

Classification thresholds:

    Compound ≥ 0.05 → Positive

    Compound ≤ -0.05 → Negative

    Between -0.05 and 0.05 → Neutral


In [10]:
# ypred = []
# from tqdm.notebook import tqdm

# for i in range(len(df)):
#     ypred.append(sia.polarity_scores(df['Text'][i]))

In [11]:
from tqdm import tqdm

dff= df.head(1000)

res={
}

for i,row in tqdm(dff.iterrows(), total=len(dff)):
    text=row['Text']
    myid=row['Id']
    res[myid]=sia.polarity_scores(text)

100%|██████████| 1000/1000 [00:00<00:00, 1644.70it/s]


    df.iterrows() generates an iterator that returns each row in the DataFrame as a tuple (index, row). The row is a pandas Series containing the data of the current row.

    tqdm(df.iterrows(), total=len(df)) wraps the iterator in the tqdm function, which adds a progress bar to show the progress as the rows are processed. The total=len(df) part ensures the progress bar shows the correct percentage of completion.


In [12]:
vaders=pd.DataFrame(res).T
vaders.head()

,neg,neu,pos,compound
1,0.000,0.695,0.305,0.9441
2,0.138,0.862,0.000,-0.5664
3,0.091,0.754,0.155,0.8265
4,0.000,1.000,0.000,0.0000
5,0.000,0.552,0.448,0.9468


extra the transfromers pipeline

In [19]:
from transformers import pipeline

In [24]:
import torch
print(torch.cuda.is_available
      (), torch.__version__)


False 2.7.0+cpu


In [26]:
sent_pipeline=pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

sent_pipeline('booo')

d:\MachineLearningJourney\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\supep\.cache\huggingface\hub\models--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.